In [9]:
import os

import librosa

# for displaying playback
import IPython.display as ipd

import tensorflow as tf
from tensorflow import keras

import tensorflow_io as tfio

# System filepath setup
import sys

# add data folders to path
sys.path.insert(1, '../data')
sys.path.insert(1, '../src')
sys.path.insert(1, '../img')

In [10]:
! cd ..

In [11]:
ls

LICENSE    README.md  data/      img/       notebooks/ src/


In [12]:
import pyaudio 
import struct
import numpy as np

import matplotlib.pyplot as plt

%matplotlib tk


import src.audio_functions as afun
import src.audio_prep as aprep

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# From tutorial:

'''
https://www.youtube.com/watch?v=AShHJdSIxkY
'''


import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))


afun.stream_seconds(how_long=1)


In [ ]:
devinfo = p.get_device_info_by_index(4)  # Or whatever device you care about.
if p.is_format_supported(48000,  # Sample rate
                         input_device=devinfo['index'],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
    print(devinfo['maxInputChannels'])
    print('Yay!')

In [ ]:
ls

In [ ]:
mel_model = tf.keras.models.load_model('data/saved_models/mel_1_sec_model_(95acc)/mel_1_sec_model')

# Confirm by checking the archetecture...
mel_model.summary()

In [ ]:
afun.record_short(1)

In [ ]:
librosa_test, sr = librosa.load('file.wav',
                                sr=22050)

In [ ]:
ipd.Audio(librosa_test, 
          rate=sr)

In [ ]:
import src.audio_functions as a_fun

In [ ]:
a_fun.record_short()

# Resources:

PyAudio
-[Insatlling PyAudio on Python 3.7](https://stackoverflow.com/questions/54998028/how-do-i-install-pyaudio-on-python-3-7)
- [Select Specific Input Device with PyAudio](https://stackoverflow.com/questions/36894315/how-to-select-a-specific-input-device-with-pyaudio)  


- [Let's Build an Audio Spectrum Analyzer in Python (pt1)](https://www.youtube.com/watch?v=AShHJdSIxkY)  

Librosa
- [Librosa with Microphone Input](https://stackoverflow.com/questions/59056786/python-librosa-with-microphone-input)  
- [Librosa Documentation](https://librosa.org/doc/latest/core.html#signal-generation)


Tensorflow
- [Tensorflow Audio Data Prep and Augmentation](https://www.tensorflow.org/io/tutorials/audio#read_an_audio_file)  
- [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) 

- [Paper: Speech Recognition](https://arxiv.org/pdf/1904.08779.pdf)



